In [10]:
from glob import glob
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np

In [11]:
def load_data(file, sheet_name):
    return pd.read_excel(file, sheet_name=sheet_name)

In [12]:
PATH = "C:/Users/withus/Desktop/sabin/data/lease/"
files = glob(PATH + '*.xlsx')

news = pd.DataFrame()
news_popular = pd.DataFrame()
twitter = pd.DataFrame()
blog = pd.DataFrame()
community = pd.DataFrame()

for file in files:
    news = pd.concat([news, load_data(file, '뉴스')], axis=0)

## 뉴스 Sheet

In [13]:
news.isna().sum()

Unnamed: 0     0
Unnamed: 1     0
Unnamed: 2     0
Unnamed: 3    15
dtype: int64

In [14]:
news.columns = news.iloc[0]
news.drop(0, inplace=True, axis=0)
news = news.dropna(axis=0, how='any')
news = news[~news['내용'].str.contains('http')]
news = news[news['내용'].str.contains('[가-힣]')]
news = news[news['내용'].str.len() < 1000]

print(news.shape)
news.head()

(3999, 4)


,작성일,작성자,제목,내용
36,2020/07/29,한국경제,"은성수 ""임대차 3법 때문에 가계부채 급증 없을 것""",국회 정무위원회 업무보고 답변 은성수 금융위원장이 29일 오전 서울 여의도 국회에서...
46,2020/11/09,한국경제TV,"""전세대란, 임대차3법도 영향"" 홍남기는 뒤늦게 인정했지만...",[한국경제TV 조현석 기자] 홍남기 경제부총리 겸 기획재정부 장관이 임대차3법이 최...
49,2020/09/28,조선비즈,"시민단체, 임대차3법 헌법소원… “늘어난 것 정부 세수뿐”",28일 서울 종로구 헌법재판소 앞에서 임대차3법 반대모임과 행동하는 자유시민 관계자...
50,2020/09/25,조선비즈,"""임대차 3법, 명백한 재산침해"" 시민단체, 헌법소원 제기",주택 임대차 보장기간을 최대 4년으로 늘리는 내용을 골자로 한 임대차3법(주택임대차...
99,2020/07/29,뉴스1,"은성수 ""임대차 3법으로 가계부채 급증하지는 않을 것""",국회 정무위원회 전체회의 은성수 금융위원장이 29일 오전 서울 여의도 국회에서 열린...


## TF-IDF

In [27]:
def tfdif(corpus):
    cv = CountVectorizer(ngram_range=(1, 2), max_features=1000)
    document_term_matrix = cv.fit_transform(corpus)

    tf = pd.DataFrame(document_term_matrix.toarray(), columns=cv.get_feature_names())  # TF (Term Frequency)

    D = len(tf)
    df = tf.astype(bool).sum(axis=0)
    idf = np.log((D+1) / (df+1)) + 1             # IDF (Inverse Document Frequency)

    # TF-IDF (Term Frequency-Inverse Document Frequency)
    tfidf = tf * idf                      
    tfidf = tfidf / np.linalg.norm(tfidf, axis=1, keepdims=True)
    
    return tfidf


In [28]:
tfidf_news = tfdif(news['내용'])

c:\Users\withus\.conda\envs\ML\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [29]:
tfidf_news

,10,10월,10일,11,12,12월,13,13일,14,16,...,홍남기 부총리,확대,회의에서,효과가,효과를,후보,후보가,후보는,후보자는,후속
0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994,0.000000,0.0,0.0,0.0,0.132036,0.0,0.0,0.0,0.0,0.0,...,0.0,0.118056,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3995,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3996,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3997,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [30]:
import pickle

tfidf_news.to_pickle('./tfidf_df/tfidf_news.pkl')